# Simple Linear versus Ridge Regression 

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [195]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold


###  Importing the dataset

In [196]:
# Import the boston dataset from sklearn
# Load dataset to some variable 
boston_data = load_boston()

In [197]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target

#  Reshape Y to be a rank 2 matrix using y.reshape()
y=y.reshape(X.shape[0],1)

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])

# Printing out the features
print('The features: ', boston_data.feature_names)

# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])

# Observing the first 2 rows of the data
print(X[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [198]:
# Create a PolynomialFeatures object with degree = 2. Using PolynomialFeatures(degree=2)
# Transform X and save it into X_2 using poly.fit_transform(X)
# Simply copy Y into Y_2 

pol = PolynomialFeatures(degree=2)
X_2 = pol.fit_transform(X)
y_2 = y

#Here I have created polynomial features which will be implemented later on in the code

In [199]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


# Your code goes here

The get_coeff_ridge_normaleq function returns the optimum weight values when the ridge term is also taken into
account whilst minimizing the least-squared loss function

In [200]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values


def get_coeff_ridge_normaleq(X_train, y_train, alpha):
 # use np.linalg.pinv(...)
    # W = ((X_T . X + alpha * I )^-1) . X_T . y
    bias_column = np.ones((X_train.shape[0],1))
    X_train = np.c_[X_train,bias_column]
    X_T = X_train.transpose()
    X_T_X = X_T.dot(X_train)

    m_1 = X_T_X + alpha * np.identity (X_T_X.shape[1])
    m_1 = np.linalg.pinv(m_1)
    m_2 = X_T.dot(y_train)
    w = m_1.dot(m_2)

    return w



The get_coeff_linear_normaleq returns the optimum weight values when least squares is used as loss function. 
Here, there is no ridge term.

In [201]:
# Define get_coeff_linear_normaleq function. Use the normal equation method.
# Return w values

def get_coeff_linear_normaleq(X_train, y_train):
    # use np.linalg.pinv(...)
    bias_column = np.ones((X_train.shape[0],1))
    X_train = np.c_[X_train,bias_column]
    X_T = X_train.transpose()
    X_T_X = X_T.dot(X_train)
    m_1 = np.linalg.pinv(X_T_X)
    m_2 = X_T.dot(y_train)
    w = m_1.dot(m_2)

    return w



The evaluate_err function returns the errors in our models predictions of the training and test test.

In [202]:
# Define the evaluate_err_ridge function.
# Return the train_error and test_error values


def evaluate_err(X_train, X_test, y_train, y_test, w): 
    #pred_train=prediction using w and X_tran+np.mean(y_train)
    #pred_test=prediction using w and X_test
    #remember to add the mean back
    bias_column = np.ones((X_train.shape[0],1))
    X_train = np.c_[X_train,bias_column]
    pred_train = X_train.dot(w)
    test_bias_column = np.ones((X_test.shape[0],1))
    X_test = np.c_[X_test,test_bias_column]  
    pred_test = X_test.dot(w) 
    total_train_error = np.square(np.subtract(y_train,pred_train))
    train_error = total_train_error.mean()  #mean squared train error
    total_test_error = np.square(np.subtract(y_test,pred_test))
    test_error = total_test_error.mean() #mean squared test error
    
    return train_error, test_error
    

The k_fold_cross_validation function randomly divides the set of observations into k groups, or folds.
The first fold is treated as a test set, and the method is fit on the remaining k − 1 folds.
The process is then repeated until each of the remaining k-1 folds has been used as a test set. Then the 
average error from the k-fold cross validation is returned

In [203]:
# Finish writting the k_fold_cross_validation function. 
# Returns the average training error and average test error from the k-fold cross validation
# Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha,regression_type):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_test_error = 0
    total_train_error = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        # Subtract y_train_mean from y_train and y_test
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        
        # Scaling the data matrix
        # Using scaler=preprocessing.StandardScaler().fit(...)
        # And scaler.transform(...)
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Determine the training error and the test error
        # Use get_coeff_linear_normaleq or get_coeff_ridge_normaleq to get w
        # And use evaluate_err()

        if regression_type == "linear":
            w_linear_matrix = get_coeff_linear_normaleq(X_train, y_train)
            current_train_error, current_test_error = evaluate_err(X_train,X_test, y_train, y_test ,w_linear_matrix)
            total_train_error = total_train_error+current_train_error
            total_test_error = total_test_error+current_test_error


        else:
            w_ridge = get_coeff_ridge_normaleq( X_train, y_train , alpha)
            current_train_error, current_test_error = evaluate_err(X_train,X_test,y_train,y_test,w_ridge)
            total_train_error = total_train_error+current_train_error
            total_test_error = total_test_error+ current_test_error

       ##############
    
    #compute average for all Ks
    avg_train_error = total_train_error/k
    avg_test_error = total_test_error/k

        
        
    return  avg_train_error,avg_test_error
    


Before we can compare the linear model with the ridge model we must find the optimum alpha for the ridge model. 
Therefore before I compare the results of the linear and ridge model I have first found the optimum alpha for the ridge model in both cases (when the dataset has no polynomial features and when it does)

In [204]:
# determining best alpha for ridge model when dataset has no polynomial features:
alpha_values = np.logspace(1, 7, num=13)
for i in alpha_values:
    avg_ridge_train_error, avg_ridge_test_error, = k_fold_cross_validation(10,X,y,i,"ridge")
    print("-"*20 +"Alpha= "+str(i)+"-"*20)
    print("Train Error: "+str(avg_ridge_train_error)+" Test Error: "+str(avg_ridge_test_error))

#from here we can see that the alpha was gives the minimum test error is alpha = 10.

--------------------Alpha= 10.0--------------------
Train Error: 21.89290115757019 Test Error: 23.68858300163877
--------------------Alpha= 31.622776601683793--------------------
Train Error: 22.2854440556975 Test Error: 24.01784029738615
--------------------Alpha= 100.0--------------------
Train Error: 23.72548838488208 Test Error: 25.293852553695142
--------------------Alpha= 316.22776601683796--------------------
Train Error: 28.16655409854012 Test Error: 29.457296222896787
--------------------Alpha= 1000.0--------------------
Train Error: 38.53214872734357 Test Error: 39.48949335939869
--------------------Alpha= 3162.2776601683795--------------------
Train Error: 54.00702649955449 Test Error: 54.64719107794949
--------------------Alpha= 10000.0--------------------
Train Error: 69.2597817132532 Test Error: 69.71851750884511
--------------------Alpha= 31622.776601683792--------------------
Train Error: 78.53074243629268 Test Error: 78.92162110435102
--------------------Alpha= 100000.

In [205]:
# determining best alpha for ridge model when dataset has polynomial features:
alpha_values = np.logspace(1, 7, num=13)
for i in alpha_values:
    avg_ridge_train_error, avg_ridge_test_error, = k_fold_cross_validation(10,X_2,y_2,i,"ridge")
    print("-"*20 +"Alpha= "+str(i)+"-"*20)
    print("Train Error: "+str(avg_ridge_train_error)+" Test Error: "+str(avg_ridge_test_error))

#from here we can see that the alpha was gives the minimum test error is alpha = 10.

--------------------Alpha= 10.0--------------------
Train Error: 10.049055874118885 Test Error: 13.476138001136226
--------------------Alpha= 31.622776601683793--------------------
Train Error: 12.751706269046771 Test Error: 15.829601969051158
--------------------Alpha= 100.0--------------------
Train Error: 16.222690593210803 Test Error: 18.98001881500993
--------------------Alpha= 316.22776601683796--------------------
Train Error: 19.700253646409855 Test Error: 22.068692308062744
--------------------Alpha= 1000.0--------------------
Train Error: 24.287457983108894 Test Error: 26.21847559440484
--------------------Alpha= 3162.2776601683795--------------------
Train Error: 33.08666831985054 Test Error: 34.58026601880018
--------------------Alpha= 10000.0--------------------
Train Error: 46.76199935941409 Test Error: 47.78952058756748
--------------------Alpha= 31622.776601683792--------------------
Train Error: 62.009001916836056 Test Error: 62.646288813443356
--------------------Alph

Now that we have calculated the optimum alpha for both the ridge models we can go ahead and compare the ridge models 
with the linear models.

Here we are comparing the performance of the linear and ridge models when the dataset has <b>no polynomial features</b>

In [206]:
# print the error for the both linear regression and ridge regression when dataset has no polynomial features
# the error should include both training error and testing error
avg_linear_train_error_nopoly, avg_linear_test_error_nopoly = k_fold_cross_validation(10,X,y,0,"linear")
print("Average Linear Train Error with no polynomial features: ", avg_linear_train_error_nopoly)
print("Average Linear Test Error with no polynomial features: ",avg_linear_test_error_nopoly)

avg_ridge_train_error_nopoly, avg_ridge_test_error_nopoly = k_fold_cross_validation(10,X,y,10,"ridge")
print("Average Ridge Train Error with no polynomial features: ", avg_ridge_train_error_nopoly)
print("Average Ridge Test Error with no polynomial features: ",avg_ridge_test_error_nopoly)

Average Linear Train Error with no polynomial features:  21.806183575851065
Average Linear Test Error with no polynomial features:  23.636068605428157
Average Ridge Train Error with no polynomial features:  21.89290115757019
Average Ridge Test Error with no polynomial features:  23.68858300163877


Here we are comparing the performance of the linear and ridge models when the dataset <b> has polynomial features</b>

In [207]:
# print the error for the both linear regression and ridge regression when dataset has polynomial features
# the error should include both training error and testing error
avg_linear_train_error_poly, avg_linear_test_error_poly, = k_fold_cross_validation(10,X_2,y_2,0,"linear")
print("Average Linear Train Error with polynomial features: ",avg_linear_train_error_poly)
print("Average Linear Test Error with polynomial features: ",avg_linear_test_error_poly)

avg_ridge_train_error_poly, avg_ridge_test_error_poly, = k_fold_cross_validation(10,X_2,y_2,10,"ridge")
print("Average Ridge Train Error with polynomial features: ",avg_ridge_train_error_poly)
print("Average Ridge Test Error with polynomial features: ",avg_ridge_test_error_poly)

Average Linear Train Error with polynomial features:  5.8088208160124655
Average Linear Test Error with polynomial features:  11.854968236414866
Average Ridge Train Error with polynomial features:  10.049055874118885
Average Ridge Test Error with polynomial features:  13.476138001136226


<h2>Summary and Evaluation</h2> 

In order to best summarise my findings I have printed out the test error of all four models.

In [208]:
print("Average Linear Test Error with no polynomial features: ",avg_linear_test_error_nopoly)
print("Average Ridge Test Error with no polynomial features: ",avg_ridge_test_error_nopoly)
print("Average Linear Test Error with polynomial features: ",avg_linear_test_error_poly)
print("Average Ridge Test Error with polynomial features: ",avg_ridge_test_error_poly)

Average Linear Test Error with no polynomial features:  23.636068605428157
Average Ridge Test Error with no polynomial features:  23.68858300163877
Average Linear Test Error with polynomial features:  11.854968236414866
Average Ridge Test Error with polynomial features:  13.476138001136226


The interpretation of my findings is quite interesting and suprising. First let us compare the ridge regression
vs linear regression. Usually one expects that because of the ridge regression, even though bias will go down, the 
error from variance will go down more and it will lead to a greater fall in error. However, this does not seem to 
be the case over here. In both instances (no polynomial features and polynomial features) the linear regression model
outperforms the ridge regression model. Next let us see no polynomial features in dataset vs polynomial features in 
dataset. After introducing polynomial features, the test error in both the linear and ridge models went down 
significantly. The reason for this may have been that the features have more of a polynomial relation with the target 
variable as compared to a basic linear one. Introducing the polynomial features helped increase the complexity of the 
model, fit the dataset better, and hence reduce the test error. All in all then I would use the linear regression 
model with polynomial features to in the future to predict housing prices as it gave the least test error.